Author: Hiranya Jayakody. April 2020.

Code developed for Smart Robotics Viticulture Group, UNSW, Sydney.

Neural Network based on Matterport implementation of Mask-RCNN at https://github.com/matterport/Mask_RCNN

### PART 1: Install Mask-RCNN repo from Matterport ###

In [0]:
!git clone https://github.com/matterport/Mask_RCNN.git
!pip install -r 'Mask_RCNN/requirements.txt'
!cd Mask_RCNN ; python setup.py install

In [0]:
!pip show mask-rcnn

In [0]:
!pip install tensorflow==1.5.1

In [0]:
!pip install keras==2.1.5

### PART 2: Set-up Mask-RCNN for training ###

In [0]:
import os
import cv2
import glob
import sys
import json
import datetime
import numpy as np
import skimage.draw
import imutils
import imgaug
import statistics as st
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from mrcnn.config import Config
from mrcnn import visualize
from mrcnn import model as modellib, utils
from matplotlib import pyplot as plt

In [0]:
#mount necessary folders
from google.colab import drive
drive.mount('/content/drive')


In [0]:
#define all directories
CWD = 'drive/My Drive/Colab Notebooks/'
STOMATA_WEIGHTS_PATH = os.path.join(CWD,'2020_mask_rcnn_stomata_51.h5')
WEIGHT_FILE_NAME = 'stomata'
CLASS_NAME = 'stomata'
DEFAULT_LOGS_DIR = os.path.join(CWD,'logs/')
DATASET_DIR = os.path.join(CWD,'images/')
TRAINING_IMG_DIR = os.path.join(DATASET_DIR,'train/')

In [0]:
# Find mean pixel value for training
DATA_PATH = os.path.join(TRAINING_IMG_DIR,'*jpg')
training_files = glob.glob(DATA_PATH)

avg_b = []
avg_g = []
avg_r = []

print(DATA_PATH)
for img in training_files:
    image = cv2.imread(img)
    b, g, r = np.average(image, axis = (0,1))
    avg_b.append(b)
    avg_g.append(g)
    avg_r.append(r)
    
mean_b = np.average(avg_b[:])
mean_g = np.average(avg_g[:])
mean_r = np.average(avg_r[:])

print([mean_b,mean_g,mean_r])

In [0]:
#create custom config class for training
class CustomConfig(Config):
    NAME = CLASS_NAME #provide a suitable name
    IMAGES_PER_GPU = 2 #set to one for smaller GPUs
    NUM_CLASSES = 1+1 #background+number of classes
    STEPS_PER_EPOCH = 100 #number of training steps per epoch
    RPN_ANCHOR_SCALES =(12,24, 48, 96, 192) 
    DETECTION_MAX_INSTANCES = 300
    DETECTION_MIN_CONFIDENCE = 0.6 
    LOSS_WEIGHTS = {'mrcnn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'mrcnn_class_loss': 1.0, 'rpn_bbox_loss': 1.0}
    IMAGE_MAX_DIM = 1024
    IMAGE_MIN_DIM = 800
    RPN_NMS_THRESHOLD = 0.9
    RPN_TRAIN_ANCHORS_PER_IMAGE = 1024
    MEAN_PIXEL = np.array([mean_r,mean_g,mean_b]) #matterport takes the input as RGB



In [0]:
#add augmentation for input data
augmentation = imgaug.augmenters.Sometimes(4/6,imgaug.augmenters.OneOf(
                                            [ 
                                            imgaug.augmenters.Affine(rotate=(-30, 30)),
                                            imgaug.augmenters.Affine(rotate=(-45, 45)), 
                                            imgaug.augmenters.Affine(rotate=(-90, 90)), 
                                            ]))


In [0]:
#create custom dataset class
class CustomDataset(utils.Dataset):
    def load_customdata(self, dataset_dir, subset):
        #subset can be either training or validation
        
        #We add the classes here.
        self.add_class(CLASS_NAME,1,CLASS_NAME) #add_class(self,source,class_id,class_name)
       
        assert subset in ['train','val']
        dataset_dir = os.path.join(dataset_dir,subset)
        
        #next step is to load the annotations
        annotations = json.load(open(os.path.join(dataset_dir,'via_region_data.json')))
        annotations = list(annotations.values())
        
        #skip unannotated images
        annotations = [a for a in annotations if a['regions']]
        
        for a in annotations:
            #get the x,y coordinates of points of the polygon
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()] 
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]
                
            # load_mask() requires the image size
            image_path = os.path.join(dataset_dir,a['filename'])
            image = skimage.io.imread(image_path)
            height,width = image.shape[:2]

            self.add_image(CLASS_NAME, image_id = a['filename'], path=image_path, width=width, height=height, polygons=polygons) #add_image(self, source, image_id, path, **kwargs)

    #override function for load_mask
    def load_mask(self, image_id):
        #this function generates instance masks for an image
        
        #if not a custom dataset image, delegate to parent class
        image_info = self.image_info[image_id]
        if image_info['source'] != CLASS_NAME:
            return super(self._class__,self).load_mask(image_id)
        
        #convert polygons to bitmap mask of shape
        info = self.image_info[image_id]
        mask = np.zeros([info['height'], info['width'],len(info['polygons'])], dtype= np.uint8)
        
        for i,p in enumerate(info['polygons']):
            rr,cc = skimage.draw.polygon(p['all_points_y'],p['all_points_x'])
            mask[rr,cc,i] = 1
            
        # return mask and class IDs. for 1 class we return 1s (CAN WE MODIFY THIS?)
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)
    
    #overrids function for image_reference
    def image_reference(self, image_id):
        #returns the path of the image
        info = self.image_info[image_id]
        
        if info['source'] == CLASS_NAME:
            return info['path']
        else:
            super(self.__class__,self).image_reference(image_id)
            

In [0]:
#set-up training function
def train(model):
    #training dataset
    dataset_train = CustomDataset()
    dataset_train.load_customdata(DATASET_DIR, 'train') 
    dataset_train.prepare()
    
    #validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_customdata(DATASET_DIR, 'val') 
    dataset_val.prepare()
    
    print('training network heads')
    #for transfer learning often training the heads layers should be enough. You can modify the number of epochs here.
    model.train(dataset_train, dataset_val, learning_rate = config.LEARNING_RATE, epochs = 40, layers='heads', augmentation = augmentation) 
    #otherwise layers = 'all' or 'heads'
    


### PART3: Execute training


In [0]:
#initialize config and model for training
config = CustomConfig()
config.display()

#create model and load default weights from stomata model
model = modellib.MaskRCNN(mode='training',config=config, model_dir= DEFAULT_LOGS_DIR)
print('loading weights', STOMATA_WEIGHTS_PATH)
model.load_weights(STOMATA_WEIGHTS_PATH, by_name=True)

In [0]:
train(model)